In [1]:
pip install azureml-sdk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pkg_resources


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pkg_resources (from versions: none)
ERROR: No matching distribution found for pkg_resources

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import azureml.core
print(azureml.core.VERSION)


ImportError: cannot import name 'Iterable' from 'collections' (c:\Users\eugene\AppData\Local\Programs\Python\Python312\Lib\collections\__init__.py)

In [4]:
# Azure ML 라이브러리 불러오기
from azureml.core import Workspace, Dataset, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.datapath import DataPath

# Azure ML 작업 영역 연결
ws = Workspace.from_config()

# 실험 생성
experiment = Experiment(workspace=ws, name='food-classification-experiment')

# GPU 컴퓨팅 클러스터 설정
compute_name = 'gpu-cluster'
if compute_name not in ws.compute_targets:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[compute_name]


ImportError: cannot import name 'Iterable' from 'collections' (c:\Users\eugene\AppData\Local\Programs\Python\Python312\Lib\collections\__init__.py)

In [10]:
!pip install torch torchvision torchaudio numpy pandas matplotlib tqdm scikit-learn pillow azureml-sdk azureml-core azureml-dataset-runtime



You should consider upgrading via the 'c:\users\eugene\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# 데이터 변환 설정 (학습 데이터 / 검증 데이터)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(), #이미지 랜덤 크롭롭
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #이미지 정규화화
    ]),
    'val': transforms.Compose([
        transforms.Resize(), 
        transforms.CenterCrop(), #이미지 크롭 크기 설정정
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #아미지 정규화화
    ]),
}

# 데이터 경로 설정 (Azure ML에서 마운트됨)
data_dir = '경로'  # Azure ML에 마운트된 데이터 경로
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


# ResNet-50 불러오기 (전이 학습)
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # 150개 음식 클래스 수에 맞게 출력층 변경
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

# 모델 학습
model = train_model(model, criterion, optimizer, num_epochs=25)

# 모델 저장
torch.save(model.state_dict(), '경로')


ModuleNotFoundError: No module named 'torch'